In [1]:
# Необходимые библиотеки
! pip install pandasql
! pip install sqlalchemy==1.4.46

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=15310f35b01e29a25deba9249137925adb06c84fa60c4ee661e40c3a98b1fe27
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.24
    Uninstalling SQLAlchemy-2.0.24:
      Successfully uninstalled SQLAlchemy-2.0.24
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.


In [14]:
import pandas as pd
from pandasql import sqldf
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
orders = pd.read_csv("/content/drive/MyDrive/sravni/orders.csv")
sales = pd.read_csv("/content/drive/MyDrive/sravni/sales.csv")
product = pd.read_csv("/content/drive/MyDrive/sravni/product.csv")
store = pd.read_csv("/content/drive/MyDrive/sravni/store.csv")

In [4]:
#функция для обновления строки в столбце
def update_string(row):
    updated_row = row.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return updated_row

#применение функции к столбцу
orders['product_info'] = orders['product_info'].apply(update_string)

print(orders)

         order_id  order_date  user_id  store_id  \
0               1  2022-01-11       13         6   
1               2  2022-01-11       70         6   
2               3  2022-01-11       75         6   
3               4  2022-01-11      172         6   
4               5  2022-01-11      176         6   
...           ...         ...      ...       ...   
1009995   1009996  2023-11-30     9966        15   
1009996   1009997  2023-11-30     9978        15   
1009997   1009998  2023-11-30     9979        15   
1009998   1009999  2023-11-30     9984        15   
1009999   1010000  2023-11-30     9995        15   

                                              product_info  
0        [{"product_id": 3283, "product_price": 18245.9...  
1        [{"product_id": 505, "product_price": 11060.84...  
2        [{"product_id": 1546, "product_price": 3065.23...  
3        [{"product_id": 3354, "product_price": 7131.35...  
4        [{"product_id": 3595, "product_price": 13762.1...  
...      

In [9]:
sales['purchase_date'].max()

'2023-12-07'

In [10]:
simple_query = ('''
SELECT
    store.store_id AS STORE_ID,
    json_extract(json_each.value, '$.product_id') AS PRODUCT_ID,
    SUM(CAST(json_extract(json_each.value, '$.count') AS INTEGER)) AS TOTAL_COUNT
FROM
    orders
CROSS JOIN
    json_each(orders.product_info)
LEFT JOIN
    sales ON sales.order_id = orders.order_id
LEFT JOIN
    product ON product.product_id = CAST(json_extract(json_each.value, '$.product_id') AS INTEGER)
LEFT JOIN
    store ON store.store_id = orders.store_id
WHERE
    sales.is_accepted = True AND sales.is_canceled = False
    AND sales.purchase_date >= date('2023-12-07', '-3 month') -- фильтр по дате
GROUP BY
    store.store_id,
    json_extract(json_each.value, '$.product_id')

''')
grouped_table = pd.DataFrame(sqldf(simple_query, env = None))
grouped_table.head()

,STORE_ID,PRODUCT_ID,TOTAL_COUNT
0,1,1,288
1,1,2,7
2,1,3,18
3,1,4,136
4,1,5,3


Подсчитаем, сколько единиц каждого товара необходимо поставить в каждый из магазинов в следующем месяце на основе статистики за три последних месяца.


In [21]:
grouped_table['AVERAGE_COUNT'] = (grouped_table['TOTAL_COUNT'] / 3)
grouped_table.head(30)

,STORE_ID,PRODUCT_ID,TOTAL_COUNT,AVERAGE_COUNT
0,1,1,288,96.000000
1,1,2,7,2.333333
2,1,3,18,6.000000
3,1,4,136,45.333333
4,1,5,3,1.000000
5,1,6,272,90.666667
6,1,7,11,3.666667
7,1,8,262,87.333333
8,1,9,27,9.000000
9,1,10,11,3.666667


In [22]:
grouped_table.to_excel('output.xlsx')

#загружаем файл
from google.colab import files
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>